In [33]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc-julia/state_constraints/Project.toml`


In [34]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using MeshCat
using ColorTypes
using GeometryBasics: HyperSphere, HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf


include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [35]:
vis = Visualizer()

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat Visualizer with path /meshcat at http://127.0.0.1:8702

In [36]:
# Quadrotor parameters
m = 0.035 # m = 0.033 <- measured m
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
gravity = 9.81
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
# kt=1
km = kt*thrustToTorque #4.4733e-8

h = 1/100

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
@show uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# @show uhover = (m*gravity/4)*ones(4) # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * gravity) / kt) / 4) * ones(4) = [0.5833333520642209, 0.5833333520642209, 0.5833333520642209, 0.5833333520642209]


In [37]:
function quad_dynamics(x,u)
  r = x[1:3]
  q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
  v = x[8:10]
  ω = x[11:13]
  Q = qtoQ(q)
  
  ṙ = v
  q̇ = 0.5*L(q)*H*ω
  
  b = 1/m

  v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 

  ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
  
  return [ṙ; q̇; v̇; ω̇]
end
function quad_dynamics_rk4(x,u)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5*h*f1, u)
  f3 = quad_dynamics(x + 0.5*h*f2, u)
  f4 = quad_dynamics(x + h*f3, u)
  xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
  xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [38]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [39]:
# Cost weights
max_dev_x = [0.1; 0.1; 0.1; 0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
max_dev_u = [0.5; 0.5; 0.5; 0.5]
Q = spdiagm(1 ./(max_dev_x.^2))
R = spdiagm(1 ./(max_dev_u.^2))
Qf = Q

# # Number of LQR direct solve matrix sets to cache
ρ_list_length = 1

# Allocate cache
cache = (
    A = Ãdyn,
    B = B̃dyn,
    # ρ_list = [diagm(ones(nu)) for _ in 1:ρ_list_length],
    ρ_list = [zeros(1) for _ in 1:ρ_list_length],
    Kinf_list = [zeros(nu,nx) for _ in 1:ρ_list_length],
    Pinf_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    Quu_inv_list = [zeros(nu,nu) for _ in 1:ρ_list_length],
    AmBKt_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    coeff_d2p_list = [zeros(nx,nu) for _ in 1:ρ_list_length],
)

# Precompute all cached matrices for multiple ρ values starting at ρ = .1 and multiplying by 5
for k = 1:ρ_list_length
    # ρ
    # ρ = diagm([1; 1; 1; 1])*1*5^(k-1)
    ρ = 5*5^(k-1);
    R̃ = R + ρ*I;
    Q̃ = Q + ρ*I;
    Q̃f = Qf + ρ*I;
    
    # Compute Kinf, Pinf
    Nriccati = 10000
    P = [zeros(nx,nx) for i = 1:Nriccati];   # cost to go quadratic term
    K = [zeros(nu,nx) for i = 1:Nriccati-1]; # feedback gain
    P[Nriccati] .= Q̃f;
    for k = (Nriccati-1):-1:1
        K[k] = (R̃ + cache.B'*P[k+1]*cache.B)\(cache.B'*P[k+1]*cache.A);
        P[k] = Q̃ + cache.A'*P[k+1]*(cache.A - cache.B*K[k]);
    end

    # Cache precomputed values
    cache.ρ_list[k] .= ρ;
    cache.Kinf_list[k] .= K[1];
    cache.Pinf_list[k] .= P[1];
    cache.Quu_inv_list[k] .= (R̃ + cache.B'*cache.Pinf_list[k]*cache.B)\I;
    cache.AmBKt_list[k] .= (cache.A - cache.B*cache.Kinf_list[k])';
    cache.coeff_d2p_list[k] .= cache.Kinf_list[k]'*R̃ - cache.AmBKt_list[k]*cache.Pinf_list[k]*cache.B;
end

# Define final time for ref trajectory
Tfinal = 15.0
N = Int(Tfinal/h)+1

# Create reference trajectory
X̃ref = [zeros(nx) for i = 1:N]
Xref = [[0; 0; 1; 1; zeros(9)] for i = 1:N]
Uref = [zeros(nu) for i = 1:N-1]

# Compute reference velocity from reference position
for i = 1:N-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))
;

In [40]:
function export_mat_to_c(declare, data)
    str = "tinytype" * declare * "= {\n"
    for i = 1:size(data, 1)
        str = str * "  "
        for j = 1:size(data, 2)
            if i == size(data, 1) && j == size(data, 2)
                this_str = @sprintf("%.7f\t", data[i, j])
            else
                this_str = @sprintf("%.7f,\t", data[i, j])
            end
            str = str * this_str
        end
        str = str * "\n"
    end
    str = str * "};"
    return str
end

function export_diag_to_c(declare, data)
    str = "tinytype" * declare * "= {"
    for i = 1:size(data, 1)
        if i == size(data, 1)
            this_str = @sprintf("%.7f", data[i, i])
        else
            this_str = @sprintf("%.7f,\t", data[i, i])
        end
        str = str * this_str
    end
    str = str * "};"
    return str
end

function export_Xref_to_c(declare, data)
    str = "tinytype" * declare * "= {\n"
    for i = 1:size(data, 1)
        str = str * "  "
        for j = 1:nx
            if i == size(data, 1) && j == nx
                this_str = @sprintf("%.7f\t", data[i][j])
            else
                this_str = @sprintf("%.7f,\t", data[i][j])
            end
            # str = str * this_str * "f"
            str = str * this_str
        end
        str = str * "\n"
    end
    str = str * "};"
    return str
end

export_Xref_to_c (generic function with 1 method)

In [42]:
boilerplate = "#pragma once\n\n#include <tinympc/types.hpp>\n\n"

rho_string = "tinytype rho_value = " * string(cache.ρ_list[1][1]) * ";\n\n"

A_data_string = export_mat_to_c(" Adyn_data[NSTATES*NSTATES] ", cache.A) * "\n\n"
B_data_string = export_mat_to_c(" Bdyn_data[NSTATES*NINPUTS] ", cache.B) * "\n\n"

Kinf_data_string = export_mat_to_c(" Kinf_data[NINPUTS*NSTATES] ", cache.Kinf_list[1]) * "\n\n"
Pinf_data_string = export_mat_to_c(" Pinf_data[NSTATES*NSTATES] ", cache.Pinf_list[1]) * "\n\n"
Quu_inv_data_string = export_mat_to_c(" Quu_inv_data[NINPUTS*NINPUTS] ", cache.Quu_inv_list[1]) * "\n\n"
AmBKt_data_string = export_mat_to_c(" AmBKt_data[NSTATES*NSTATES] ", cache.AmBKt_list[1]) * "\n\n"
coeff_d2p_data_string = export_mat_to_c(" coeff_d2p_data[NSTATES*NINPUTS] ", cache.coeff_d2p_list[1]) * "\n\n"

Q_data_string = export_diag_to_c(" Q_data[NSTATES]", Q) * "\n\n"
Qf_data_string = export_diag_to_c(" Qf_data[NSTATES]", Qf) * "\n\n"
R_data_string = export_diag_to_c(" R_data[NINPUTS]", R) * "\n\n"

file = open("quadrotor_100hz_params.hpp", "w")
write(file, boilerplate);
write(file, rho_string);
write(file, A_data_string);
write(file, B_data_string);
write(file, Kinf_data_string);
write(file, Pinf_data_string);
write(file, Quu_inv_data_string);
write(file, AmBKt_data_string);
write(file, coeff_d2p_data_string);
write(file, Q_data_string);
write(file, Qf_data_string);
write(file, R_data_string);

SystemError: SystemError: opening file "quadrotor_100hz_params.hpp": No such file or directory

In [45]:
X̃_ref_string = export_Xref_to_c(" Xref_data[NTOTAL*NSTATES] ", X̃ref)

open("quadrotor_100hz_ref_hover.hpp", "w") do file
    write(file, boilerplate)
    write(file, X̃_ref_string)
end

202675